In [ ]:
import csv
from functools import reduce
import json

# Fungsi untuk memvalidasi apakah string dapat dikonversi ke float
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Baca data dari file CSV dan filter
data_list = []
with open('/content/911_Calls_for_Service_(Last_30_Days).csv', mode='r') as file:
    reader = csv.DictReader(file)
    data_list = [row for row in reader]

# Filter dictionary yang memiliki kolom Zip dan Neighborhood serta kolom waktu yang valid
filtered_data = list(filter(lambda x: x['zip_code'] and x['neighborhood'] and is_float(x['totalresponsetime']) and is_float(x['dispatchtime']) and is_float(x['totaltime']), data_list))

# Fungsi untuk menghitung rata-rata waktu
def calculate_averages(data):
    total_respons_time = reduce(lambda acc, x: acc + float(x['totalresponsetime']), data, 0)
    total_dispatch_time = reduce(lambda acc, x: acc + float(x['dispatchtime']), data, 0)
    total_time = reduce(lambda acc, x: acc + float(x['totaltime']), data, 0)
    count = len(data)

    return {
        'average_response_time': total_respons_time / count if count else 0,
        'average_dispatch_time': total_dispatch_time / count if count else 0,
        'average_total_time': total_time / count if count else 0
    }

averages = calculate_averages(filtered_data)


In [ ]:
from itertools import groupby

# Pisahkan data berdasarkan Neighborhood
filtered_data.sort(key=lambda x: x['neighborhood'])
neighborhood_groups = {k: list(v) for k, v in groupby(filtered_data, key=lambda x: x['neighborhood'])}

# Fungsi untuk menghitung rata-rata waktu untuk setiap Neighborhood
def calculate_neighborhood_averages(neighborhood_data):
    total_respons_time = reduce(lambda acc, x: acc + float(x['totalresponsetime']), neighborhood_data, 0)
    total_dispatch_time = reduce(lambda acc, x: acc + float(x['dispatchtime']), neighborhood_data, 0)
    total_time = reduce(lambda acc, x: acc + float(x['totaltime']), neighborhood_data, 0)
    count = len(neighborhood_data)

    return {
        'average_response_time': total_respons_time / count if count else 0,
        'average_dispatch_time': total_dispatch_time / count if count else 0,
        'average_total_time': total_time / count if count else 0
    }

neighborhood_averages = [
    {'Neighborhood': neighborhood, **calculate_neighborhood_averages(data)}
    for neighborhood, data in neighborhood_groups.items()
]

# Menyertakan data populasi untuk seluruh Detroit
combined_data = neighborhood_averages + [averages]


In [ ]:
# Simpan hasil ke file JSON
with open('output.json', 'w') as json_file:
    json.dump(combined_data, json_file, indent=4)

In [ ]:
import json

# Membaca file JSON
with open('output.json', 'r') as json_file:
    data = json.load(json_file)

# Menampilkan konten dari file JSON
print(json.dumps(data, indent=4))


[
    {
        "Neighborhood": "Airport Sub",
        "average_response_time": 15.299305555555556,
        "average_dispatch_time": 9.606249999999996,
        "average_total_time": 41.40069444444445
    },
    {
        "Neighborhood": "Arden Park",
        "average_response_time": 3.1,
        "average_dispatch_time": 0.425,
        "average_total_time": 15.274999999999999
    },
    {
        "Neighborhood": "Aviation Sub",
        "average_response_time": 18.947826086956525,
        "average_dispatch_time": 10.945652173913045,
        "average_total_time": 48.373913043478254
    },
    {
        "Neighborhood": "Bagley",
        "average_response_time": 13.171982758620686,
        "average_dispatch_time": 8.239655172413794,
        "average_total_time": 30.209913793103446
    },
    {
        "Neighborhood": "Barton-McFarland",
        "average_response_time": 23.510067114093953,
        "average_dispatch_time": 13.956375838926165,
        "average_total_time": 63.54899328859059
  